# Laboratorio 7 - Ataques a modelos de Deep Learning

In [13]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import keras
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from art.attacks.extraction import CopycatCNN
from art.attacks.evasion import AdversarialPatch
from art.estimators.classification import KerasClassifier
from keras.models import load_model
from sklearn.utils import shuffle

# Disabling eager execution from TF 2
tf.compat.v1.disable_eager_execution()

In [2]:
def load_data():
    DIRECTORY = os.getcwd()
    DIRECTORY = os.path.join(DIRECTORY, "malimg_paper_dataset_imgs")
    images = []
    labels = []
    i = 0
    for folder in os.listdir(DIRECTORY):
        folder_directory = os.path.join(DIRECTORY, folder)
        if not os.path.isdir(folder_directory): continue
                
        for file in os.listdir(folder_directory):
            img_path = os.path.join(folder_directory, file)
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (150, 150))
            labels.append(i)
            images.append(image)
            
        i += 1
    
    images = np.array(images, dtype = 'float32')
    labels = np.array(labels, dtype = 'int32')
    
    return images, labels

In [3]:
target_model = load_model('modelo_lab6')
classifier = KerasClassifier(model=target_model, clip_values=(0,1))

2023-05-22 23:16:54.281428: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:353] MLIR V1 optimization pass is not enabled
2023-05-22 23:16:54.324196: W tensorflow/c/c_api.cc:300] Operation '{name:'AssignVariableOp_2' id:148 op device:{requested: '/device:CPU:0', assigned: ''} def:{{{node AssignVariableOp_2}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false, _device="/device:CPU:0"](count_1, Identity_2)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


In [4]:
images, labels = load_data()

In [5]:
n_images = 1000
images, labels = shuffle(images[:n_images], labels[:n_images], random_state=123)

In [6]:
breakpoint = int(n_images*0.7)
train_images = images[:breakpoint]
test_images = images[breakpoint:]
test_labels = labels[breakpoint:]

In [9]:
def create_blank_model():
    model = tf.keras.models.Sequential([
        Conv2D(32, (3, 3), activation="relu", input_shape=(150, 150, 3)),
        MaxPool2D(2, 2),
        Conv2D(32, (3, 3), activation="relu"),
        MaxPool2D(2, 2),
        Flatten(),
        Dense(128, activation=tf.nn.relu),     
        Dense(25, activation=tf.nn.softmax)
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
        )

    return model

## Ataque de extracción (copycat)

In [7]:
copycat = CopycatCNN(classifier=classifier)

In [8]:
copycat.batch_size_fit = 128
copycat.batch_size_query = 128
copycat.nb_epochs = 6
copycat.nb_stolen = n_images

In [10]:
model_stolen = KerasClassifier(
    model=create_blank_model(),
    clip_values=(0,1)
)

In [11]:
stolen_classifier = copycat.extract(
    train_images,
    thieved_classifier=model_stolen
)

2023-05-22 23:18:19.181953: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_1/Softmax' id:227 op device:{requested: '', assigned: ''} def:{{{node dense_1/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_1/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-22 23:18:19.225701: W tensorflow/c/c_api.cc:300] Operation '{name:'conv2d_2/bias/Assign' id:367 op device:{requested: '', assigned: ''} def:{{{node conv2d_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](conv2d_2/bias, conv2d_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Train on 700 samples
Epoch 1/6


2023-05-22 23:18:20.941663: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_1/mul' id:507 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-22 23:18:21.005933: W tensorflow/c/c_api.cc:300] Operation '{name:'training/Adam/dense_2/kernel/v/Assign' id:714 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_2/kernel/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_2/kernel/v, training/Adam/dense_2/kernel/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the futu

700/700 [==============================] - 4s 6ms/sample - loss: 727.4578 - accuracy: 0.2100
Epoch 2/6
700/700 [==============================] - 4s 6ms/sample - loss: 43.6055 - accuracy: 0.5757
Epoch 3/6
700/700 [==============================] - 4s 6ms/sample - loss: 0.2663 - accuracy: 0.9500
Epoch 4/6
700/700 [==============================] - 4s 6ms/sample - loss: 0.1473 - accuracy: 0.9700
Epoch 5/6
700/700 [==============================] - 4s 6ms/sample - loss: 0.0367 - accuracy: 0.9900
Epoch 6/6
700/700 [==============================] - 4s 5ms/sample - loss: 0.0079 - accuracy: 1.0000


In [12]:
# Testing the performance of the original classifier
score_original = classifier._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Testing the performance of the stolen classifier
score_stolen = stolen_classifier._model.evaluate(
    x=test_images, 
    y=test_labels
    )

# Comparing test losses
print(f"Original test loss: {score_original[0]:.4f} " 
      f"vs stolen test loss: {score_stolen[0]:.4f}")

# Comparing test accuracies
print(f"Original test accuracy: {score_original[1]:.4f} " 
      f"vs stolen test accuracy: {score_stolen[1]:.4f}")

2023-05-22 23:18:45.256289: W tensorflow/c/c_api.cc:300] Operation '{name:'loss/mul' id:295 op device:{requested: '', assigned: ''} def:{{{node loss/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss/mul/x, loss/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-05-22 23:18:45.880842: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_1/mul' id:507 op device:{requested: '', assigned: ''} def:{{{node loss_1/mul}} = Mul[T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul/x, loss_1/dense_1_loss/value)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Original test loss: 0.1371 vs stolen test loss: 0.3321
Original test accuracy: 0.9867 vs stolen test accuracy: 0.9767


## Ataque de evasión (Adversarial Patch)

In [ ]:
attack = AdversarialPatch(classifier, rotation_max=22.5, scale_min=0.1, scale_max=1.0, learning_rate=5.0, max_iter=500)